In [1]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref
import ast
import random
from tqdm import tqdm_notebook

depeches = pd.read_csv("depeches.csv")
photos = pd.read_csv("photos.csv",encoding='latin-1')

c:\users\maison\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
c:\users\maison\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
C:\Users\Maison\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,15,17,23,24,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
nlp=spacy.load("en_core_web_md")  
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

In [ ]:
# Décommenter votre partie
#ton_chiffre = 1 # Dimitri
#ton_chiffre = 2 # Louis
#ton_chiffre = 3 # Yannis
#ton_chiffre = 4 # Yohan

In [ ]:
nlp_photos = []
#for i in tqdm_notebook(range(100000)): # Dimitri
#    nlp_photos.append(nlp(photos.caption[i]))
#for i in tqdm_notebook(range(100000,200000)): # Louis
#    nlp_photos.append(nlp(photos.caption[i]))
#for i in tqdm_notebook(range(200000,300000)): # Yannis
#    nlp_photos.append(nlp(photos.caption[i]))
#for i in tqdm_notebook(range(300000, photos.shape[0])): # Yohan
#    nlp_photos.append(nlp(photos.caption[i]))

C:\Users\Maison\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [ ]:
def mentioned(ent, cluster):
    for mention in cluster.mentions:
        if(ent.text in mention.text):
            return(True)
ents_dict = []
for document in nlp_photos:
    dictionary = {}
    for ent in document.ents:
        dep_list = []
        start = ent.start
        end = ent.end
        for i in range(start,end):
            word = document[i]
            if(word.head.text not in ent.text and (word.dep_ == 'compound' or word.dep_ == 'conj')):
                while(word.dep_ == 'compound' or word.dep_ == 'conj'):
                    word = word.head
                dep_list.append(word.dep_)
            else:
                dep_list.append(word.dep_)
        if(ent.text in dictionary.keys()):
            dictionary[ent.text] += dep_list
        else:
            dictionary[ent.text] = dep_list
    ents_dict.append(dictionary)
corefs_dict = []
for document in nlp_photos:
    dictionary = {}
    clusters = document._.coref_clusters
    for ent in document.ents:
        corefs_list = []
        for cluster in clusters:
            if(mentioned(ent,cluster)):
                for mention in cluster.mentions:
                    mention_deps = []
                    start = mention.start
                    end = mention.end
                    for i in range(start,end):
                        word = document[i]
                        if(word.head.text not in mention.text and (word.dep_ == 'compound' or word.dep_ == 'conj')):
                            while(word.dep_ == 'compound' or word.dep_ == 'conj'):
                                word = word.head
                            mention_deps.append(word.dep_)
                        else:
                            mention_deps.append(word.dep_)
                    corefs_list.append((mention.text, mention_deps))     
        if(ent.text not in dictionary.keys()):
            dictionary[ent.text] = corefs_list
    corefs_dict.append(dictionary)
labels_dict = []
for document in nlp_photos:
    dictionary = {}
    for ent in document.ents:
        if(ent.text not in dictionary.keys()):
            dictionary[ent.text] = [ent.label_]
        else:
            dictionary[ent.text] += [ent.label_]
    labels_dict.append(dictionary)
final_dataframe = pd.DataFrame({'Labels' : labels_dict, 'Ents' : ents_dict, 'Corefs' : corefs_dict})
final_dataframe.to_pickle('final_dataframe_photos'+'_'+str(ton_chiffre)+'.csv')